In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [69]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.mnist.load_data()

In [57]:
from sklearn.preprocessing import LabelBinarizer
encoded=LabelBinarizer().fit(y_train)
y_train=encoded.fit_transform(y_train)
y_test=encoded.fit_transform(y_test)

In [5]:
y_train

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0]])

In [6]:
y_test

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [7]:
learning_rate=0.00001
epochs=10
batch_size=128

In [72]:
x.shape

TensorShape([Dimension(None), Dimension(28), Dimension(28), Dimension(1)])

In [9]:
y_test.shape

(10000, 10)

In [59]:
x_train.dtype,x_test.dtype,y_test.dtype,y_train.dtype='float32','float32','float32','float32'


In [11]:
assert x_train.dtype=='float32'

In [12]:
x_train.dtype

dtype('float32')

In [13]:
validation_size=256

In [14]:
n_classes=10

In [15]:
dropout=0.75

In [44]:
weights={
    'wc1':tf.Variable(tf.random_normal([5,5,1,32])),
    'wc2':tf.Variable(tf.random_normal([5,5,32,64])),
    'wd1':tf.Variable(tf.random_normal([7*7*64,1024])),
    'out':tf.Variable(tf.random_normal([1024, n_classes]))
}

In [45]:
biases={
    'bc1':tf.Variable(tf.zeros([32])),
    'bc2':tf.Variable(tf.zeros([64])),
    'bd1':tf.Variable(tf.zeros([1024])),
    'out':tf.Variable(tf.zeros([n_classes]))
}

In [42]:
def conv2d(x, W, b, Strides):
    con=tf.nn.conv2d(x,W,strides=[1,Strides,Strides,1],padding='SAME')
    con=tf.nn.bias_add(con, b)
    return tf.nn.relu(con)

In [39]:
def maxpool2d(x, k):
    x=tf.nn.max_pool(x, ksize=[1,k,k,1],strides=[1,k,k,1],padding='SAME')
    

In [40]:
def conv_net(x, weights, biases, dropout):
    #28*28-->28*28*32
    conv1=conv2d(x, weights['wc1'], biases['bc1'],1)
    #28*28*32-->14*14*32
    conv1_pooled=maxpool2d(conv1,2)
    
    #14*14*64
    conv2=conv2d(conv1_pooled,weights['wc2'],biases['bc2'],1)
    #14*14*64-->7*7*64
    conv2_pooled=maxpool2d(conv2,2)
    
    
    conv2_reshaped=tf.reshape(conv2_pooled,[-1,weights['wd1'].get_shape().as_list()[0]])
    fc1=tf.add(tf.multiply(conv2_reshaped,weights['wd1']),biases['bd1'])
    fc1=tf.nn.relu(fc1)
    fc1=tf.nn.dropout(fc1,dropout)
    
    out=tf.add(tf.matmul(fc1,weights['out']),biases['out'])
    return out
    

In [21]:
weights['wd1'].get_shape()

TensorShape([Dimension(3136), Dimension(1024)])

In [22]:
 [-1, weights['wd1'].get_shape().as_list()[0]]

[-1, 3136]

In [23]:
weights['wd1'].shape

TensorShape([Dimension(3136), Dimension(1024)])

In [73]:
x=tf.placeholder("float32",[None,28,28,1])
y=tf.placeholder("float32", [None, n_classes])
keep_prob=tf.placeholder("float32")

logits=conv_net(x, weights, biases, keep_prob)

cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=y))
optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

pred=tf.equal(tf.argmax(logits,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(pred,tf.float32))

init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(epochs):
        total_data=x_train.shape[0]
        batches=total_data/batch_size
        i=0
        while i<batches:
            
            if i==int(batches):
                batch_x,batch_y=x_train[i*batch_size:total_data],y_train[i*batch_size:total_data]
                
            else:
                batch_x,batch_y=x_train[i*batch_size:i*batch_size+batch_size],y_train[i*batch_size:i*batch_size+batch_size]
            
            sess.run(optimizer,feed_dict={x:batch_x,y:batch_y,keep_prob: dropout})
            
            #calc loss and accuracy
            loss=sess.run(cost,feed_dict={x:batch_x,y:batch_y,keep_prob:1.})
            
            valid_acc=sess.run(accuracy,feed_dict={x:x_test[:validation_size],y:y_test[:validation_size],keep_prob:1.})
            
            print('Epoch {:>2}, Batch {:>3} -'
                  'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                i + 1,
                loss,
                valid_acc))
    test_acc=sess.run(accuracy, feed_dict={x:x_test[validation_size:validation_size*2],y:y_test[validation_size:validation_size*2],keep_prob: 1.}) 
    print('Testin accuracy :{}'.format(test_acc))
            
            
            
       

AttributeError: 'NoneType' object has no attribute 'shape'